# Laboratorul 5 

## Introducere
Acest document prezintă o aplicație complexă dezvoltată în Python, concepută pentru procesarea și manipularea imaginilor folosind biblioteci moderne de grafică și editare vizuală. Aplicația oferă o interfață grafică modernă și intuitivă pentru manipularea imaginilor cu următoarele caracteristici principale:

1. Suport pentru diverse spații de culoare (RGB, Grayscale, HSV etc.)
2. Gestionarea și conversia între multiple formate de imagine (BMP, JPEG, PNG etc.)
3. Implementarea unor tehnici de editare a imaginilor, precum ajustarea luminozității, contrastului și aplicarea filtrelor
4. Instrumente pentru scalare și rotație eficientă a imaginilor
5. Interfață grafică prietenoasă și ușor de utilizat pentru manipularea diverselor funcționalități

Proiectul demonstrează aplicarea practică a conceptelor fundamentale de procesare a imaginilor, oferind o platformă utilă pentru explorarea tehnicilor de editare și analiză vizuală.

## Structura Codului

Aplicatia este organizata in urmatoarele sectiuni principale:

1. Instalarea modulelor necesare
2. Declararea variabilelor globale ale aplicației de laborator
3. Clase pentru vizualizare (Spectrograma, Informatii Audio, Spectru de Frecventa)
4. Functii pentru gestionarea fisierelor audio
5. Functii pentru redare si control
6. Efecte audio (filtre si efecte creative)
7. Procesarea si aplicarea efectelor
8. Functii pentru inregistrare
9. Initializare UI
10. Exercitii (TODO-uri)
11. Initializarea aplicatiei

## 1. Instalarea modulelor necesare
Din pricina faptului că sunt folosite anumite module care nu sunt prevăzute în mod implicit în Python Vanilla, va trebui să le importăm, lucru care se poate realiza prin rularea codului de mai jos

In [177]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *

## Rolul modulelor pe care le vom folosi
Python este un limbaj de programare care se folosește foarte mult de module, așa că este momentul potrivit pentru a explica în mare ce face fiecare dintre modulele pe care le-am importat în blocul de deasupra.
Astfel:
* **sys** și **os** – Are ca rol manipularea sistemului și facilitează interacțiunea cu fișierele/directoarele sistemului de calcul.
* **cv2** – OpenCV este unul dintre tool-urile principale care ne vor ajuta la procesarea de imagini.
* **numpy as np** – Numpy este folosit pentru a facilita manipularea mult mai eficientă a matricelor și datelor numerice.
* **PyQt5.QtWidgets** – Acest modul ne ajută la realizarea componentelor pentru GUI.
* **PyQt5.QtGui** – Modulul ne ajută la gestionarea imaginilor, fonturilor și pictogramelor în cadrul GUI-ului.
* **PyQt5.QtCore** – Acest modul prezintă funcționalități de bază precum gestionarea evenimentelor și a thread-urilor în cadrul programului.


## 2. Declararea variabilelor globale ale aplicației de laborator

Pentru a putea dispune de o lizibilitate mai bună a codului, dar de asemenea și de o organizare mai bună, am considerat definirea unor variabile globale. Acest motiv vine din faptul că ...

In [178]:
# Variabile globale pentru starea aplicației

current_image=None 
original_image=None
current_path=None
app=None
main_window=None
image_label=None
brightness_slider=None
contrast_slider=None
filter_combo=None
smoothing_slider=None
scale_combo=None
rotation_combo=None
format_combo=None
tabs=None
format_tab=None
selected_format=".png"

In [179]:
def load_image():
    """Încarcă o imagine din sistemul de fișiere."""
    global current_image, original_image, current_path
    
    file_dialog = QFileDialog()
    file_path, _ = file_dialog.getOpenFileName(main_window, "Încarcă imagine", "", "Fișiere imagine (*.png *.jpg *.jpeg *.bmp *.webp)")
    
    if file_path:
        current_path = os.path.abspath(file_path)
        original_image = cv2.imread(current_path, cv2.IMREAD_UNCHANGED)
        if original_image is None:
            return
        
        current_image = original_image.copy()
        update_image_display()

In [180]:
def update_image_display():
    """Actualizează afișarea imaginii în interfață."""
    global current_image, image_label
    
    if current_image is None:
        return
        
    # Convertește imaginea din BGR la RGB (OpenCV folosește BGR)
    rgb_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)
    
    height, width, channels = rgb_image.shape
    bytes_per_line = channels * width
    q_image = QImage(rgb_image.data, width, height, bytes_per_line, QImage.Format_RGB888)
    
    pixmap = QPixmap.fromImage(q_image)
    image_label.setPixmap(pixmap)
    image_label.adjustSize()

In [181]:
def change_color_space(index):
    """Schimbă spațiul de culoare al imaginii."""
    global current_image, original_image,computed,using_cv2
    
    if original_image is None:
        return
        
    current_image = original_image.copy()
    
    if index == 0:  # Original
        pass
    elif index == 1:  # Grayscale computed
        current_image=np.array(0.299 * original_image[:,:,2] + 0.587 *original_image[:,:,1] + 0.114*original_image[:,:,0]).astype(np.uint8)
        computed=current_image.copy()
        pass
    elif index == 2:  # Grayscale
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)
        using_cv2=current_image.copy()
    elif index==3:
         diff=np.abs(using_cv2-computed)
         diff=cv2.normalize(diff,None,0,255,cv2.NORM_MINMAX)
         current_image=diff
         if(np.sum(current_image)):
             print("Cele 2 conversii nu sunt perfect identice")
         else:
             print("Cele 2 conversii sunt identice")
    elif index == 4:  # HSV
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2HSV)
    elif index == 5: # HSV computed
         hsv_computed=current_image.astype(np.float32)/255.0
         r,g,b = original_image[...,0], original_image[...,1], original_image[...,2]
         c_max= np.max(original_image,axis=-1)
         c_min= np.min(original_image,axis=-1)
         delta=c_max-c_min

         h=np.zeros_like(c_max,dtype=np.float32)
         mask=delta > 0
         r_mask = (c_max==r) & mask
         g_mask = (c_max==g) & mask
         b_mask = (c_max==b) & mask

         h[r_mask]= (60 * ((g[r_mask] - b[r_mask]) / delta[r_mask]) + 360) % 360
         h[g_mask]= (60 * ((b[g_mask] - r[g_mask]) / delta[g_mask]) + 120) % 360
         h[b_mask]= (60 * ((r[b_mask] - g[b_mask]) / delta[b_mask]) + 240) %360
         h[ h<0 ]+= 360

         s = np.zeros_like(c_max,dtype=np.float32)
         s[c_max > 0] = delta[c_max > 0] / c_max[c_max > 0]
         v = c_max
         h = (h / 2).astype(np.uint8)  # H in OpenCV is 0-179
         s = (s * 255).astype(np.uint8)
         v = (v * 255).astype(np.uint8)

         current_image=np.stack([h, s, v], axis=-1)
    elif index == 6:  # HSL
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2HLS)
    elif index == 7: # HSL Computed
         hsl_computed=current_image.astype(np.float32)/255.0
         r,g,b= original_image[...,0], original_image[...,1], original_image[...,2]
         cmax=np.max(original_image,axis=-1)
         cmin=np.min(original_image,axis=-1)
         delta=cmax-cmin
         l = (cmax + cmin) / 2

    # Compute Saturation (S)
         s = np.zeros_like(l)
         mask = delta > 0
         s[mask] = delta[mask] / (1 - np.abs(2 * l[mask] - 1))
    
        # Compute Hue (H)
         h = np.zeros_like(cmax,dtype=np.float32)
         r_mask = (cmax == r) & mask
         g_mask = (cmax == g) & mask
         b_mask = (cmax == b) & mask
    
         h[r_mask] = (60 * ((g[r_mask] - b[r_mask]) / delta[r_mask]) % 6)
         h[g_mask] = (60 * ((b[g_mask] - r[g_mask]) / delta[g_mask]) + 2)
         h[b_mask] = (60 * ((r[b_mask] - g[b_mask]) / delta[b_mask]) + 4)
         h[h < 0] += 360  # Ensure positive values
    
        # Scale H, S, L for OpenCV-like representation:
         h = (h / 2).astype(np.uint8)  # Scale H to [0, 179]
         s = (s * 255).astype(np.uint8)  # Scale S to [0, 255]
         l = (l * 255).astype(np.uint8)  # Scale L to [0, 255]

         current_image=np.stack([h,s,l],axis=-1)
    elif index == 8:  # YUV
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2YUV)
    elif index == 9:  # YCrCb
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2YCrCb)
    elif index == 10:  # CMYK 
        bgr = current_image.astype(float)/255.0
        k = 1 - np.max(bgr, axis=2)
        c = (1-bgr[...,2] - k)/(1-k+0.00001)
        m = (1-bgr[...,1] - k)/(1-k+0.00001)
        y = (1-bgr[...,0] - k)/(1-k+0.00001)
        
        # Convertim înapoi la BGR pentru afișare
        cmyk = np.zeros_like(current_image, dtype=np.float32)
        cmyk[...,0] = 255 * c
        cmyk[...,1] = 255 * m
        cmyk[...,2] = 255 * y
        current_image = cmyk.astype(np.uint8)
    elif index == 11:  # LAB
        current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2LAB)
        
    update_image_display()

In [182]:
def save_image():
    """Salvează imaginea curentă în formatul selectat."""
    global current_image, selected_format
    
    if current_image is None or not selected_format:
        return
    
    file_dialog = QFileDialog()
    file_path, _ = file_dialog.getSaveFileName(main_window, "Salvează imaginea", "", f"Fișiere imagine (*{selected_format})")
    
    if file_path:
        # Asigură-te că extensia este corectă
        if not file_path.endswith(selected_format):
            file_path += selected_format
        
        cv2.imwrite(file_path, current_image)

In [183]:
def adjust_brightness_contrast():
    """Ajustează luminozitatea și contrastul imaginii."""
    global current_image, original_image, brightness_slider, contrast_slider
    
    if original_image is None:
        return
        
    brightness = brightness_slider.value()
    contrast = contrast_slider.value() / 100.0
    
    current_image = original_image.copy()
    
    # Aplicare brightness și contrast
    current_image = cv2.convertScaleAbs(current_image, alpha=contrast, beta=brightness)
    
    update_image_display()

In [184]:
def apply_filter(index):
    """Aplică un filtru selectat pe imagine."""
    global current_image, original_image
    
    if original_image is None:
        return
        
    current_image = original_image.copy()
    
    if index == 0:  # Fără filtru
        pass
    elif index == 1:  # Blur
        current_image = cv2.blur(current_image, (5, 5))
    elif index == 2:  # Gaussian Blur
        current_image = cv2.GaussianBlur(current_image, (5, 5), 0)
    elif index == 3:  # Median Blur
        current_image = cv2.medianBlur(current_image, 5)
    elif index == 4:  # Bilateral Filter
        current_image = cv2.bilateralFilter(current_image, 9, 75, 75)
        
    update_image_display()

In [185]:
def show_histogram():
    """Afișează histograma imaginii curente."""
    global current_image
    
    if current_image is None:
        return
        
    # Creează histograma
    gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    
    # Creează o imagine pentru histogramă
    hist_img = np.zeros((300, 256, 3), np.uint8)
    cv2.normalize(hist, hist, 0, 300, cv2.NORM_MINMAX)
    
    for i in range(256):
        cv2.line(hist_img, (i, 300), (i, 300 - int(hist[i])), (255, 255, 255), 1)
        
    # Afișează histograma într-o fereastră nouă
    cv2.imshow("Histograma", hist_img)

In [186]:
def apply_smoothing():
    """Aplică smoothing (netezire) pe imagine."""
    global current_image, original_image, smoothing_slider
    
    if original_image is None:
        return
        
    kernel_size = smoothing_slider.value()
    if kernel_size % 2 == 0:
        kernel_size += 1  # Kernel size trebuie să fie impar
        
    current_image = original_image.copy()
    current_image = cv2.GaussianBlur(current_image, (kernel_size, kernel_size), 0)
    
    update_image_display()

In [187]:
def apply_sharpening():
    """Aplică sharpening (accentuare) pe imagine."""
    global current_image, original_image
    
    if original_image is None:
        return
        
    kernel = np.array([[-1, -1, -1],
                      [-1, 9, -1],
                      [-1, -1, -1]])
                      
    current_image = original_image.copy()
    current_image = cv2.filter2D(current_image, -1, kernel)
    
    update_image_display()

In [188]:
def apply_scaling():
    """Aplică scalare (redimensionare) pe imagine."""
    global current_image, original_image, scale_combo
    
    if original_image is None:
        return
        
    scale_index = scale_combo.currentIndex()
    scales = [1.0, 0.5, 0.25, 2.0, 4.0]
    
    if scale_index < 0 or scale_index >= len(scales):
        return
        
    scale = scales[scale_index]
    
    # Calculează noile dimensiuni
    height, width = original_image.shape[:2]
    new_width = int(width * scale)
    new_height = int(height * scale)
    
    # Redimensionează imaginea
    current_image = cv2.resize(original_image, (new_width, new_height), interpolation=cv2.INTER_AREA if scale < 1 else cv2.INTER_LINEAR)
    
    update_image_display()

In [189]:
def apply_rotation():
    """Aplică rotație pe imagine."""
    global current_image, original_image, rotation_combo
    
    if original_image is None:
        return
        
    rotation_index = rotation_combo.currentIndex()
    angles = [0, 90, 180, 270]
    
    if rotation_index < 0 or rotation_index >= len(angles):
        return
        
    angle = angles[rotation_index]
    
    # Obține dimensiunile imaginii
    height, width = original_image.shape[:2]
    center = (width // 2, height // 2)
    
    # Calculează matricea de rotație
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Aplică rotația
    current_image = cv2.warpAffine(original_image, rotation_matrix, (width, height))
    
    update_image_display()

In [190]:
def reset_image():
    """Resetează imaginea la original."""
    global current_image, original_image, brightness_slider, contrast_slider, filter_combo, smoothing_slider
    
    if original_image is None:
        return
        
    current_image = original_image.copy()
    update_image_display()
    
    # Resetează și controalele
    brightness_slider.setValue(0)
    contrast_slider.setValue(100)
    filter_combo.setCurrentIndex(0)
    smoothing_slider.setValue(1)

In [191]:
def create_ui():
    """Creează interfața aplicației și returnează fereastra principală."""
    global app, main_window, image_label, brightness_slider, contrast_slider
    global filter_combo, smoothing_slider, scale_combo, rotation_combo, format_combo
    global tabs, format_tab, control_layout

    # Inițializare aplicație
    app = QApplication(sys.argv)
    main_window = QMainWindow()
    main_window.setWindowTitle('Editor de Imagini')
    main_window.setGeometry(100, 100, 1200, 800)

    # Widget principal
    main_widget = QWidget()
    main_window.setCentralWidget(main_widget)

    # Layout principal
    main_layout = QHBoxLayout()
    main_widget.setLayout(main_layout)

    # Splitter pentru a redimensiona zonele
    splitter = QSplitter(Qt.Horizontal)
    main_layout.addWidget(splitter)

    # Zona de imagine
    image_scroll = QScrollArea()
    image_scroll.setWidgetResizable(True)
    image_label = QLabel()
    image_label.setAlignment(Qt.AlignCenter)
    image_label.setMinimumSize(200,200)
    image_scroll.setWidget(image_label)
    splitter.addWidget(image_scroll)

    # Zona de control
    control_widget = QWidget()
    control_layout = QVBoxLayout()
    control_widget.setLayout(control_layout)
    control_widget.setMaximumWidth(400)
    splitter.addWidget(control_widget)

    # Setare proporții inițiale pentru splitter
    splitter.setSizes([800, 400])

    # Buton pentru încărcarea imaginilor
    load_button = QPushButton('Încarcă imagine')
    load_button.clicked.connect(load_image)
    control_layout.addWidget(load_button)

    # Tab widget pentru categorii
    tabs = QTabWidget()
    control_layout.addWidget(tabs)

    return main_window

In [192]:
def create_color_space_tab():
    """Creează tab-ul pentru spații de culoare și îl adaugă la interfață."""
    global tabs
    
    # 1. Spații de culoare
    color_space_tab = QWidget()
    color_space_layout = QVBoxLayout()
    color_space_tab.setLayout(color_space_layout)

    color_space_buttons = [
        ('Original', 0), ('Grayscale Computed', 1), ('Grayscale', 2),('Computed vs openCV',3),('HSV', 4),
        ('HSV Computed',5),
        ('HSL', 6), ('HSL Computed',7),('YUV', 8), ('YCrCb', 9), ('CMYK', 10), ('LAB', 11)
    ]

    for text, index in color_space_buttons:
        button = QPushButton(text)
        button.clicked.connect(lambda _, i=index: change_color_space(i))
        color_space_layout.addWidget(button)

    tabs.addTab(color_space_tab, "1. Spații de culoare")

In [193]:
def create_format_tab():
    """Creează tab-ul pentru formate de imagine și îl adaugă la interfață."""
    global tabs, format_tab, format_buttons
    
    # 2. Formate de imagine
    format_tab = QWidget()
    format_layout = QVBoxLayout()
    format_tab.setLayout(format_layout)

    format_label = QLabel("Salvează imaginea în formatul:")
    format_label.setMaximumHeight(10)
    format_layout.addWidget(format_label)

    formats = {"BMP": ".bmp", "JPEG": ".jpg", "PNG": ".png", "WEBP": ".webp", "SVG": ".svg"}
    format_buttons = {}

    for fmt, ext in formats.items():
        button = QPushButton(fmt)
        button.clicked.connect(lambda checked, e=ext: set_selected_format(e))
        format_layout.addWidget(button)
        format_buttons[ext] = button

    save_button = QPushButton("Salvează imaginea")
    save_button.clicked.connect(save_image)
    format_layout.addWidget(save_button)

    tabs.addTab(format_tab, "2. Formate de imagine")

In [194]:
def create_edit_tab():
    """Creează tab-ul pentru editare imagini și îl adaugă la interfață."""
    global tabs, brightness_slider, contrast_slider, filter_combo, smoothing_slider
    
    # 3. Editare imagini
    edit_tab = QWidget()
    edit_layout = QVBoxLayout()
    edit_tab.setLayout(edit_layout)

    # Brightness control
    brightness_group = QGroupBox("Brightness")
    brightness_layout = QVBoxLayout()
    brightness_group.setLayout(brightness_layout)

    brightness_slider = QSlider(Qt.Horizontal)
    brightness_slider.setMinimum(-100)
    brightness_slider.setMaximum(100)
    brightness_slider.setValue(0)
    brightness_slider.setTickPosition(QSlider.TicksBelow)
    brightness_slider.setTickInterval(10)
    brightness_slider.valueChanged.connect(adjust_brightness_contrast)
    brightness_layout.addWidget(brightness_slider)

    edit_layout.addWidget(brightness_group)

    # Contrast control
    contrast_group = QGroupBox("Contrast")
    contrast_layout = QVBoxLayout()
    contrast_group.setLayout(contrast_layout)

    contrast_slider = QSlider(Qt.Horizontal)
    contrast_slider.setMinimum(0)
    contrast_slider.setMaximum(200)
    contrast_slider.setValue(100)
    contrast_slider.setTickPosition(QSlider.TicksBelow)
    contrast_slider.setTickInterval(10)
    contrast_slider.valueChanged.connect(adjust_brightness_contrast)
    contrast_layout.addWidget(contrast_slider)

    edit_layout.addWidget(contrast_group)

    # Filters
    filter_group = QGroupBox("Filtre")
    filter_layout = QVBoxLayout()
    filter_group.setLayout(filter_layout)

    filter_combo = QComboBox()
    filter_combo.addItems(["Fără filtru", "Blur", "Gaussian Blur", "Median Blur", "Bilateral Filter"])
    filter_combo.currentIndexChanged.connect(apply_filter)
    filter_layout.addWidget(filter_combo)

    edit_layout.addWidget(filter_group)

    # Histogram
    histogram_button = QPushButton("Afișează histograma")
    histogram_button.clicked.connect(show_histogram)
    edit_layout.addWidget(histogram_button)

    # Smoothing
    smoothing_group = QGroupBox("Smoothing")
    smoothing_layout = QVBoxLayout()
    smoothing_group.setLayout(smoothing_layout)

    smoothing_slider = QSlider(Qt.Horizontal)
    smoothing_slider.setMinimum(1)
    smoothing_slider.setMaximum(25)
    smoothing_slider.setValue(1)
    smoothing_slider.setTickPosition(QSlider.TicksBelow)
    smoothing_slider.setTickInterval(2)
    smoothing_slider.valueChanged.connect(apply_smoothing)
    smoothing_layout.addWidget(smoothing_slider)

    edit_layout.addWidget(smoothing_group)

    # Sharpening
    sharpening_button = QPushButton("Aplică sharpening")
    sharpening_button.clicked.connect(apply_sharpening)
    edit_layout.addWidget(sharpening_button)

    tabs.addTab(edit_tab, "3. Editare imagini")

In [195]:
def create_transform_tab():
    """Creează tab-ul pentru transformări și îl adaugă la interfață."""
    global tabs, scale_combo, rotation_combo
    
    # 4. Transformări
    transform_tab = QWidget()
    transform_layout = QVBoxLayout()
    transform_tab.setLayout(transform_layout)

    # Scaling
    scaling_group = QGroupBox("Scalare")
    scaling_layout = QVBoxLayout()
    scaling_group.setLayout(scaling_layout)

    scale_combo = QComboBox()
    scale_combo.addItems(["100%", "25%", "50%", "200%", "400%"])
    scaling_layout.addWidget(scale_combo)

    apply_scale_button = QPushButton("Aplică scalare")
    apply_scale_button.clicked.connect(apply_scaling)
    scaling_layout.addWidget(apply_scale_button)

    transform_layout.addWidget(scaling_group)

    # Rotation
    rotation_group = QGroupBox("Rotație")
    rotation_layout = QVBoxLayout()
    rotation_group.setLayout(rotation_layout)

    rotation_combo = QComboBox()
    rotation_combo.addItems(["0°", "90°", "180°", "270°"])
    rotation_layout.addWidget(rotation_combo)

    apply_rotation_button = QPushButton("Aplică rotație")
    apply_rotation_button.clicked.connect(apply_rotation)
    rotation_layout.addWidget(apply_rotation_button)

    transform_layout.addWidget(rotation_group)

    tabs.addTab(transform_tab, "4. Transformări")

In [196]:
def finalize_ui():
    """Adaugă elementele finale la interfață."""
    global control_layout
    
    # Reset button
    reset_button = QPushButton("Resetează imaginea")
    reset_button.clicked.connect(reset_image)
    control_layout.addWidget(reset_button)

In [197]:
def initialize_ui():
    """Inițializează și pregătește întreaga interfață a aplicației."""
    window = create_ui()
    create_color_space_tab()
    create_format_tab()
    create_edit_tab()
    create_transform_tab()
    finalize_ui()
    return window

In [198]:
def run_app():
    """Pornește aplicația."""
    window = initialize_ui()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    run_app()

SystemExit: 0

## Concluzie si Bibliografie
Acum că ați ajuns la final, cam acesta este proiectul nostru interactiv care prezintă atât principalele spații de culori folosite în industrie, cât șo diferite formate în care imaginile sunt salvate în computere, dar și diferite operații de bază în procesarea de imagini. Bineînțeles, functionalitatile pot fi extinse prin implementarea exercițiilor propuse spre rezolvare sau prin adaugarea unor noi funcționaități care să dezvolte aplicația. Proiectul oferă o bază solida pentru înțelegerea atât a spațiilor de culoare, a formatelor de imagini, dar și a operațiilor de prelucrare a imaginilor.
Bibliografie:

* **PyQt5** Documentation: https://riverbankcomputing.com/software/pyqt/intro
* **NumPy** Documentation: https://numpy.org/doc/stable/
* **OpenCV** Documentation: https://docs.opencv.org/4.x/index.html
* Cartea **Practical Python and OpenCV** by Adrian Rosebrock https://minhtn1.github.io/Practical%20Python%20and%20OpenCV,%203rd%20Edition.pdf
